In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [127]:
#Initiation of data

#cosl = np.cos(sample_l)
#cosb = np.cos(sample_b)
#sinl = np.sin(sample_l)
#sinb = np.sin(sample_b)

#mul=sample values
#mub=sample values

nx,ny,nz = 30,30,30
hx,hy,hz = 10,10,10
phi = np.zeros((nx,ny,nz)) # First test

bvals = np.array([10.,10.,10.])
lvals = np.array([30.,30.,30.])

b = np.deg2rad(bvals) # just a test
l = np.deg2rad(lvals)
cosl = np.cos(l)
cosb = np.cos(b)
sinl = np.sin(l)
sinb = np.sin(b)
s = 0.1

S = np.array([[-sinl,cosl,0],
              [-sinb*cosl,-sinb*sinl,cosb]])

mul = np.array([1,1,1])
mub = np.array([1,1,1])

pvals = s*np.array([-sinl*cosb*mul - cosl*sinb*mub,
                 cosl*cosb*mul - sinl*sinb*mub,
                 cosb*mub])
    
rhatvals = np.array([cosb*cosl, cosb*sinl, sinb])

pvals = pvals.T
rhatvals = rhatvals.T
    
pmean = np.mean(pvals, axis=0)

In [128]:
def calc_K(pk,rhat,vxmin,dvx,nx,vymin,dvy,ny,vzmin,dvz,nz) :
    '''Calculate the values of K simultaneously for all bins'''
    K = np.zeros((nx,ny,nz))
    vmin = np.array([vxmin,vymin,vzmin])
    dvx, dvy, dvz = np.array([dvx,dvy,dvz])
    
    vxmax, vymax, vzmax = vxmin+nx*dvx,vymin+ny*dvy,vzmin+nz*dvz
    # Find intersections along one dimension (say x)
    
    vx_bins = np.arange(vxmin, vxmax+dvx, dvx)
    vy_bins = np.arange(vymin, vymax+dvy, dvy)
    vz_bins = np.arange(vzmin, vzmax+dvz, dvz)
    
    #v_space = np.array([vx_bins,vy_bins,vz_bins])
    
    pkx, pky, pkz = pk
    
    if np.linalg.norm(rhat)!=1:
        raise ValueError('rhat must be a unit vector')

    rhatx, rhaty, rhatz = rhat
    
    vrx = (vx_bins-pkx)/rhatx
    vry = (vy_bins-pky)/rhaty
    vrz = (vz_bins-pkz)/rhatz
    
    vrmax = min(max(vrx),max(vry),max(vrz))
    vrmin = max(min(vrx),min(vry),min(vrz))
    
    vrx = vrx[(vrx<=vrmax) & (vrx>=vrmin)]
    vry = vry[(vry<=vrmax) & (vry>=vrmin)]
    vrz = vrz[(vrz<=vrmax) & (vrz>=vrmin)]
    vr = np.concatenate((vrx,vry,vrz))
    #vr = np.sqrt(vrx**2+vry**2+vrz**2)
    vr.sort()
    
    vr_prime =(vr[:-1] + vr[1:]) / 2
    line_bins = np.zeros((len(vr_prime),3))
    for i in range(len(vr_prime)):
        v_prime = pk + vr_prime[i]*rhat
        line_bins[i] += np.floor((v_prime-vmin)/ dv)
    
    
    line_bins = line_bins.astype(int)
    #line_bins, unique_ind = np.unique(line_bins,axis=0,return_index=True)
    
    #line_len = vr[1:]-vr[:-1]
    line_len = vr[1:]-vr[:-1]
    non_zero = np.nonzero(line_len)
    line_len = line_len[non_zero]
    line_bins = line_bins[non_zero]
    #line_len = line_len[unique_ind]
    
    K[line_bins[:,0],line_bins[:,1],line_bins[:,2]] = line_len/(dvx*dvy*dvz)
    
    return K, line_bins, line_len, vr

In [129]:
def calc_sigma(rhatvals,pvals,pmean):
    
    rhatvals_outer = rhatvals[:,:,None]*rhatvals[:,None,:]

    iden = np.identity(3)
    
    A = np.stack([iden]*len(rhatvals_outer))-rhatvals_outer

    Ainv = np.linalg.inv(A)
    
    Ainv_mean = np.mean(Ainv,axis=0)
    v_mean = np.dot(Ainv_mean, pmean)
    
    pp = pvals - np.dot(A,v_mean)
    
    ppx, ppy, ppz = pp[:,0], pp[:,1], pp[:,2]
    
    ppx2mean = np.mean(ppx*ppx,axis=0)
    ppy2mean = np.mean(ppy*ppy,axis=0)
    ppz2mean = np.mean(ppz*ppz,axis=0)
    
    pp2mean = np.array([ppx2mean,ppy2mean,ppz2mean])
    
    A = np.array([[9,-1,-1],[-1,9,-1],[-1,-1,9]])
    
    sigma2 = (3/14)*np.dot(A,pp2mean)
    
    return sigma2

array([ 0.00051415,  0.08137924,  0.0386433 ])

In [234]:
def nl_delta(n,l):
    
    """Checks if our given vector n is within one unit vector e_i of the cell l"""
    
    e_x = np.array([1,0,0])
    e_y = np.array([0,1,0])
    e_z = np.array([0,0,1])
    
    rules = [np.array_equal(n,l+e_x),
            np.array_equal(n,l-e_x),
            np.array_equal(n,l+e_y),
            np.array_equal(n,l-e_y),
            np.array_equal(n,l+e_z),
            np.array_equal(n,l-e_z)]
    
    if np.array_equal(n,l):
        delta = -2
    elif any(rules):
        delta = 1
    else:
        delta = 0
    
    return delta
        
def calc_xhi(line_bins,sigma2,hx,hy,hz,nx,ny,nz):

    #Given a vector l, find the estimate of the second derivative. Compare l with possible adjacent n values.    
    
    h2 = np.array([hx**2,hy**2,hz**2])
    
    xhi = np.zeros((len(line_bins),7))
    
    n_bins = np.array([nx,ny,nz])
    
    e_x = np.array([1,0,0])
    e_y = np.array([0,1,0])
    e_z = np.array([0,0,1])
    
    for i in range(len(line_bins)):
        
        l = line_bins[i]
        
        n_list = [l,
                 l+e_x,l-e_x,
                 l+e_y,l-e_y,
                 l+e_z,l-e_z]
        
        print(n_list)
        
        for j in range(7):
            
            n = n_list[j]
            
            if (all(n>=0)) and (all(n<=n_bins)):
                xhi[i][j] += np.sum((sigma2/h2) *  nl_delta(n,l))

    return xhi

In [264]:
nx, ny, nz = 5, 5, 5

phi = np.ones((nx,ny,nz))

def sec_der(phi,sigma2,hx,hy,hz,nx,ny,nz):
    
    """Attempt to solve the problem without for-loops. Still needs some work"""
    
    nx, ny, nz = len(phi[0]), len(phi[1]), len(phi[2])
    
    nxx, nyy, nzz = nx+2, ny+2, nz+2
    
    h2 = np.array([hx*hx,hy*hy,hz*hz])

    phip = np.zeros((nxx,nyy,nzz))

    
    phip[1:-1,1:-1,1:-1] = phi
    
    kappa = sigma2/h2
    
    kappa_sum = sum(sigma2/h2)
    
    phi_fac = np.array([phip[0:nxx-2:,:,:]+phip[2:nxx,:,:],
                           phip[:,0:nyy-2,:]+phip[:,2:nyy,:],
                           phip[:,:,0:nzz-2]+phip[:,:,2:nzz]])
    
    phi_arr = np.dot(kappa,phi_fac)-2*kappa_sum*phip
    
    phi_arr = np.sum(phi_arr)
    
    return phi_arr